# Step 1: mechanics with Euler-Lagrange

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

from numpy import logical_and as npand
from numpy import logical_or as npor
import matplotlib

In [ ]:
from sympy import (symbols, pi, I, E, cos, sin, exp, tan, simplify, expand, factor, collect,
                   apart, cancel, expand_trig, diff, Derivative, Function, integrate, limit,
                   series, Eq, solve, dsolve, Matrix, N, preorder_traversal, Float, solve_linear_system,
                   eye, zeros, lambdify, Symbol,hessian)
from sympy.physics.mechanics import dynamicsymbols, init_vprinting

In [ ]:
from matplotlib import animation, rc
import matplotlib.patches as patches
from matplotlib.transforms import Affine2D
from IPython.display import HTML

In [ ]:
matplotlib.rcParams['animation.embed_limit'] = 200

In [ ]:
init_vprinting()

In [ ]:
m0, m1, m2, l1, l2, t, g, k = symbols('m_0 m_1 m_2 l_1 l_2 t g k')

In [ ]:
q0, q1, q2 = dynamicsymbols('q_0 q_1 q_2')
q0, q1, q2

In [ ]:
cart_x_1 = q0 + l1*sin(q1)
cart_y_1 = -l1 * cos(q1)
cart_x_2 = cart_x_1 + l2*sin(q2)
cart_y_2 = cart_y_1 - l2*cos(q2)

In [ ]:
cart_x_1_dot = cart_x_1.diff(t)
cart_y_1_dot = cart_y_1.diff(t)
cart_x_2_dot = cart_x_2.diff(t)
cart_y_2_dot = cart_y_2.diff(t)

In [ ]:
cart_v_1_sq = cart_x_1_dot**2 + cart_y_1_dot**2
cart_v_2_sq = cart_x_2_dot**2 + cart_y_2_dot**2

In [ ]:
T = (m0 * q0.diff(t)**2 + m1*cart_v_1_sq + m2*cart_v_2_sq)/2
T = simplify(T)
T

In [ ]:
U = g * (m1 * cart_y_1 + m2 * cart_y_2) + k/2*(q1-q2)**2
U

In [ ]:
from optibot.symbolic import get_lagr_eqs, lagr_to_RHS, print_funcs

In [ ]:
lag_eqs = get_lagr_eqs(T, U, 3)
lag_eqs

In [ ]:
RHS = lagr_to_RHS(lag_eqs)
RHS

In [ ]:
q_0, q_1, q_2, v_0, v_1, v_2, u_0, u_1, u_2 = symbols('q_0 q_1 q_2 v_0 v_1 v_2 u_0 u_1 u_2')

In [ ]:
T_mod = simplify(T.subs([
    [q2, q1],
    [q2.diff(t), q1.diff(t)],
    [u_2, 0],
    [m1, 0]
]))
T_mod

In [ ]:
l3 = symbols('l_3')

In [ ]:
cart_x_1_mod = q0 + (l3)*sin(q1)
cart_y_1_mod = -(l3) * cos(q1)
cart_x_1_dot_mod = cart_x_1_mod.diff(t)
cart_y_1_dot_mod = cart_y_1_mod.diff(t)
cart_v_1_sq_mod = cart_x_1_dot_mod**2 + cart_y_1_dot_mod**2
T_mod_b = (m0 * q0.diff(t)**2 + m2*cart_v_1_sq_mod)/2
T_mod_b = simplify(T_mod_b)
T_mod_b

In [ ]:
simplify(T_mod - T_mod_b)

In [ ]:
U_mod = g * (m2 * cart_y_1_mod)
U_mod

In [ ]:
hessian(T_mod_b, [q0.diff(t),q1.diff(t)])

In [ ]:
lag_eqs_mod = get_lagr_eqs(T_mod_b, U_mod, 2)
lag_eqs_mod

In [ ]:
RHS_mod = lagr_to_RHS(lag_eqs_mod)
RHS_mod

In [ ]:
from sympy.physics.mechanics import LagrangesMethod, Lagrangian

In [ ]:
Lagrangian()

In [ ]:
LM_ej = LagrangesMethod(T_mod_b - U_mod, [q0, q1])

In [ ]:
simplify(LM_ej.form_lagranges_equations())

In [ ]:
LM_ej.mass_matrix

In [ ]:
from optibot.symbolic import lagrange, diff_to_symb
from optibot.numpy import unpack

In [ ]:
T_mod_b-U_mod

In [ ]:
lagr_0 = lagrange(T_mod_b - U_mod, q0)
lagr_1 = lagrange(T_mod_b - U_mod, q1)
lagr_0, lagr_1

In [ ]:
print_funcs([diff_to_symb(lagr_0, 2),], 2)

In [ ]:
print_funcs([diff_to_symb(lagr_1, 2),], 2)

In [ ]:
def lagr_f_simp_0(x, params, ):
    q_0, q_1, v_0, v_1 = unpack(x)
    a_0, a_1, l_3, m_0, m_2 = params
    result = a_0*m_0 + a_0*m_2 + a_1*l_3*m_2*np.cos(q_1) - l_3*m_2*v_1**2*np.sin(q_1)

    return result

def lagr_f_simp_1(x, params, ):
    q_0, q_1, v_0, v_1 = unpack(x)
    a_0, a_1, g, l_3, m_2 = params
    result = a_0*l_3*m_2*np.cos(q_1) + a_1*l_3**2*m_2 + g*l_3*m_2*np.sin(q_1)

    return result



## Casadi

In [ ]:
import casadi as cas

In [ ]:
from optibot.casadi import RHS2casF

In [ ]:
from optibot.numpy import RHS2numpy, trapz_integrate, num_derivative

In [ ]:
from optibot.schemes import euler_restr, trapz_restr, trapz_mod_restr, hs_mod_restr, hs_restr
from optibot.casadi import restriction2casadi

In [ ]:
from optibot.schemes import integrate_euler, integrate_hs, integrate_trapz, integrate_rk4

### Cartpole simple

In [ ]:
def create_anim_simp(X, U, params):
    g, l_3, m_0, m_2 = params
    N = X.shape[0]
    fig, ax = plt.subplots()
    
    y_scale = l_3 * 1.25
    min_x_cart = np.min(X[:,0])
    max_x_cart = np.max(X[:,0])
    cart_displ = max_x_cart-min_x_cart
    size_x = 2*y_scale + cart_displ
    size_y = 2*y_scale
    draw_width = 14
    draw_height = draw_width / size_x * size_y
    
    x_0 = X[:,0]
    y_0 = np.zeros_like(x_0)
    x_1 = x_0 + l_3*np.sin(X[:,1])
    y_1 = y_0 - l_3*np.cos(X[:,1])
    
    x_cm = (m_0 * x_0 + m_2 * x_1)/(m_0 + m_2)
    y_cm = (m_0 * y_0 + m_2 * y_1)/(m_0 + m_2)

    fig.set_size_inches([draw_width,draw_height])
    ax.set_xlim(( min_x_cart-y_scale, max_x_cart+y_scale))
    ax.set_ylim(( -y_scale, y_scale))

    circle1 = plt.Circle((0, 0), l_3, color='b', ls = ":", fill=False)
    ax.add_artist(circle1)
    ax.plot([min_x_cart - l_3, max_x_cart + l_3], [0,0], 'k', lw=1, ls = ':')

    line1, = ax.plot([], [], lw=2)
    line3, = ax.plot([], [], 'k', lw=1, ls = ':')
    line_cm, = ax.plot([], [], 'g', lw=1, ls = ':')
    point0, = ax.plot([], [], marker='o', markersize=15, color="red")
    point1, = ax.plot([], [], marker='o', markersize=15, color="red")
    point_cm, = ax.plot([], [], marker='o', markersize=10, color="green")
    u_max = max(np.max(np.abs(U[:,0])),1e-15)
    arrow_w = 0.05*l_3
    arrow_l = 0.2*l_3
    u_arrow = patches.Arrow(0, 0, 0, -arrow_l, color = 'gray',width = arrow_w)
    ax.add_patch(u_arrow)
    
    print_vars = [X[:,0], X[:,1], U[:,0], U[:,1], np.linspace(0, N-1, N, dtype=int)]
    print_var_names = ['q_0', 'q_1', 'u_0', 'u_1', 'step']
    texts = []
    ii = 0
    for arr in print_vars:
        texts.append(ax.text(0.2, ii, "", fontsize = 12))
        ii -= 0.08*l_3
    
    def init():
        line1.set_data([], [])
        line3.set_data([], [])
        line_cm.set_data([], [])
        point1.set_data([], [])
        circle1.center = (0, 0)
        return (line1,)
    def animate(i):
        circle1.center = (x_0[i], y_0[i])
        point0.set_data(x_0[i], y_0[i])
        line1.set_data([x_0[i], x_1[i]], [y_0[i], y_1[i]])    
        point1.set_data(x_1[i], y_1[i])
        point_cm.set_data(x_cm[i], y_cm[i])
        line3.set_data(x_1[:i], y_1[:i])
        line_cm.set_data(x_cm[:i], y_cm[:i])
        trans = Affine2D()
        u_arrow._patch_transform = trans.scale(U[i,0] * arrow_l / u_max, arrow_w).translate(x_0[i],0)
        for ii in range(len(texts)):
            text = texts[ii]
            name = print_var_names[ii]
            arr = print_vars[ii]
            text.set_text("$" + name + "$ = %.6f" % arr[i])
        return (line1,u_arrow)
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=N, interval=20, 
                               blit=True)
    return anim

In [ ]:
F_cas_simp = RHS2casF(RHS_mod, 2)

In [ ]:
F_cas_simp([1,1,1,1], [1,1], [1,1,1,1])

In [ ]:
F_nump_simp = RHS2numpy(RHS_mod, 2)

In [ ]:
F_nump_simp([1,1,1,1], [1,1], [1,1,1,1])

In [ ]:
N = 600

In [ ]:
new_X_simp = np.zeros([N+1,4],dtype = 'float')
new_X_simp[:,1]= np.linspace(0, cas.pi, N+1)
new_X_simp[:,3]= cas.pi/N
new_U_simp = np.zeros([N+1,2],dtype = 'float')

In [ ]:
opti = cas.Opti()
opti.solver('ipopt')

In [ ]:
X = opti.variable(N+1,4)
U = opti.variable(N+1,2)
T = opti.variable()
u_m = opti.parameter(2)

In [ ]:
#M = opti.parameter(2)
#L = opti.parameter(2)
#G = opti.parameter()
Params = opti.parameter(4)

In [ ]:
#cost = cas.sum1(2+cas.cos(X[:,0])+cas.cos(X[:,1]))*T
#cost = cas.sum1(cas.cos(X[:,1])) + cas.sum1(X[:,0]**2)
#cost = cas.sum1(cas.cos(X[:,1]))
cost = T
opti.minimize(cost)

In [ ]:
opti.subject_to(X[0,:].T == [0, 0, 0, 0])
opti.subject_to(opti.bounded(-0.0001,X[-1,0],0.0001))
opti.subject_to(cas.cos(X[-1,1]) < -0.9999)
opti.subject_to(opti.bounded(-0.0001,X[-1,2],0.0001))
opti.subject_to(opti.bounded(-0.0001,X[-1,3],0.0001))
opti.subject_to(U[-1,:].T == [0, 0])

In [ ]:
#restriction = restriction2casadi(euler_restr, F_cas, 2, 4)
restriction = restriction2casadi(trapz_restr, F_cas_simp, 2, 4)
#restriction = restriction2casadi(hs_restr, F_cas, 2, 4)

In [ ]:
for ii in range(N):
    opti.subject_to(restriction(X[ii,:], X[ii+1,:], U[ii,:], U[ii+1,:],T/N, Params)==0)
    opti.subject_to(opti.bounded(-u_m[0],U[ii, 0],u_m[0]))
    opti.subject_to(opti.bounded(-u_m[1],U[ii, 1],u_m[1]))

In [ ]:
opti.set_initial(X[:,1], np.linspace(0, cas.pi, N+1))
opti.set_initial(X[:,3], cas.pi/N)
opti.set_initial(U, np.zeros([N+1,2],dtype = 'float'))

In [ ]:
opti.set_initial(X, new_X_simp)
opti.set_initial(U, new_U_simp)

In [ ]:
opti.set_initial(T, 12)
max_par_1 = 0.0
max_f = 1.0
opti.set_value(u_m, [max_f, max_par_1])

m_0, m_2 = [0.2, 0.4]
l_3 = 0.3
g = 9.8

opti.set_value(Params, [g, l_3, m_0, m_2])

In [ ]:
sol = opti.solve()

In [ ]:
xx_simp = sol.value(X)
uu_simp = sol.value(U) 

In [ ]:
new_X_simp = xx_simp
new_U_simp = uu_simp

In [ ]:
anim_simp = create_anim_simp(xx_simp,uu_simp,[g, l_3, m_0, m_2])

In [ ]:
HTML(anim_simp.to_jshtml())

In [ ]:
sol.value(T)/N

In [ ]:
compr_sol_simp = integrate_trapz(
    xx_simp[0,:],
    uu_simp,
    F_nump_simp,
    sol.value(T)/N,
    [g, l_3, m_0, m_2]
)

In [ ]:
compr_anim_simp = create_anim_simp(np.array(compr_sol_simp)[:-1,:],uu_simp, [g, l_3, m_0, m_2])

In [ ]:
HTML(compr_anim_simp.to_jshtml())

lagr_f_simp_0(compr_sol_simp[324], [a_0, a_1, l_3, m_0, m_2], )


lagr_f_simp_1(compr_sol_simp[324], [a_0, a_1, g, l_3, m_2], )

### Interpolation

In [ ]:
from optibot.schemes import interpolated_array

In [ ]:
from optibot.numpy import trapz_integrate, num_derivative

In [ ]:
new_t_array = np.linspace(0, sol.value(T), 2*N)

In [ ]:
h = sol.value(T)/N

old_t_array = np.linspace(0, (xx_simp.shape[0] - 1) * h, xx_simp.shape[0])

In [ ]:
new_X_interp, new_U_interp = interpolated_array(xx_simp, uu_simp, F_nump_simp, sol.value(T)/N,
                                  new_t_array, [g, l_3, m_0, m_2], 'trapz')

In [ ]:
accelerations = F_nump_simp(xx_simp, uu_simp,[g, l_3, m_0, m_2])
accelerations

In [ ]:
aa = 2
plt.figure(figsize=[50,20])
plt.plot(old_t_array,np.array(compr_sol_simp)[:-1,aa]/2, label = f'soluc integrada con trapz, componente{aa}')
plt.plot(old_t_array,accelerations[:,aa]/40, label = f'aceleraciones, componente{aa}')
plt.plot(old_t_array,accelerations[:,0]/2, label = f'aceleraciones, componente 0')
#plt.plot(old_t_array,np.array(compr_sol_simp)[:-1,0], label = f'soluc integrada, componente 0')
#plt.plot(new_t_array,new_X_interp[:,aa], label = f'soluc interpolada, componente{aa}')
plt.plot([0, old_t_array[-1] + l_3], [0,0], 'k', lw=1, ls = ':')
plt.legend()

In [ ]:
aa = 2
plt.figure(figsize=[50,20])
plt.plot(old_t_array,np.array(compr_sol_simp)[:-1,aa], label = f'soluc integrada con trapz, componente{aa}')
plt.plot(old_t_array,xx_simp[:,aa], label = f'soluc de Casadi, componente{aa}')
plt.plot(old_t_array,xx_simp[:,0], label = f'soluc de Casadi, componente 0')
plt.plot(new_t_array,new_X_interp[:,aa], label = f'soluc interpolada, componente{aa}')
plt.legend()

In [ ]:
plt.figure(figsize=[50,20])
plt.plot(new_t_array[900:1000], new_X_interp[900:1000,0], 'b')
plt.plot(old_t_array[450:500], np.array(compr_sol_simp)[450:500,0], 'r:')
plt.plot(old_t_array[450:500], np.array(compr_sol_simp)[450:500,0], 'ko')

In [ ]:
plt.figure(figsize=[50,20])
aa = 2
plt.plot(new_t_array[100:200], new_X_interp[100:200,aa], 'b')
plt.plot(old_t_array[50:100], np.array(compr_sol_simp)[50:100,aa], 'r:')
plt.plot(old_t_array[50:100], xx_simp[50:100,aa], 'g:')
plt.plot(old_t_array[50:100], np.array(compr_sol_simp)[50:100,aa], 'ko')

### Cartpole con articulación elástica

In [ ]:
def create_anim(X, U, params):
    g, k, l_1, l_2, m_0, m_1, m_2 = params
    l_3 = l_1 + l_2
    N = X.shape[0]
    fig, ax = plt.subplots()
    
    y_scale = (l_3) * 1.25
    min_x_cart = np.min(X[:,0])
    max_x_cart = np.max(X[:,0])
    cart_displ = max_x_cart-min_x_cart
    size_x = 2*y_scale + cart_displ
    size_y = 2*y_scale
    draw_width = 14
    draw_height = draw_width / size_x * size_y
    
    x_0 = X[:,0]
    y_0 = np.zeros_like(x_0)
    x_1 = x_0 + l_1*np.sin(X[:,1])
    y_1 = y_0 - l_1*np.cos(X[:,1])
    x_2 = x_1 + l_2*np.sin(X[:,2])
    y_2 = y_1 - l_2*np.cos(X[:,2])
    
    x_cm = (m_0 * x_0 + m_1 * x_1 + m_2 * x_2)/(m_0 + m_1 + m_2)
    y_cm = (m_0 * y_0 + m_1 * y_1 + m_2 * y_2)/(m_0 + m_1 + m_2)

    fig.set_size_inches([draw_width,draw_height])
    ax.set_xlim(( min_x_cart-y_scale, max_x_cart+y_scale))
    ax.set_ylim(( -y_scale, y_scale))


    circle1 = plt.Circle((0, 0), l_3, color='b', ls = ":", fill=False)
    circle2 = plt.Circle((0, 0), l_1, color='b', ls = ":", fill=False)
    ax.add_artist(circle1)
    ax.add_artist(circle2)
    ax.plot([min_x_cart - l_1-l_2, max_x_cart + l_3], [0,0], 'k', lw=1, ls = ':')

    line1, = ax.plot([], [], lw=2)
    line2, = ax.plot([], [], lw=2)
    line3, = ax.plot([], [], 'k', lw=1, ls = ':')
    line_cm, = ax.plot([], [], 'g', lw=1, ls = ':')
    point0, = ax.plot([], [], marker='o', markersize=15, color="red")
    point1, = ax.plot([], [], marker='o', markersize=15, color="red")
    point2, = ax.plot([], [], marker='o', markersize=15, color="red")
    point_cm, = ax.plot([], [], marker='o', markersize=10, color="green")
    u_max = max(np.max(np.abs(U[:,0])),1e-15)
    arrow_w = 0.05*l_3
    arrow_l = 0.2*l_3
    u_arrow = patches.Arrow(0, 0, 0, -arrow_l, color = 'gray',width = arrow_w)
    ax.add_patch(u_arrow)
    
    print_vars = [X[:,0], X[:,1], X[:,2], U[:,0], U[:,1], U[:,2], np.linspace(0, N-1, N, dtype=int)]
    print_var_names = ['q_0', 'q_1', 'q_2', 'u_0', 'u_1', 'u_2', 'step']
    texts = []
    ii = 0
    for arr in print_vars:
        texts.append(ax.text(0.2, ii, "", fontsize = 12))
        ii -= 0.08*l_3
    
    def init():
        line1.set_data([], [])
        line2.set_data([], [])
        line3.set_data([], [])
        line_cm.set_data([], [])
        point1.set_data([], [])
        point2.set_data([], [])
        circle1.center = (0, 0)
        circle2.center = (0, 0)
        return (line1,line2,)
    def animate(i):
        circle1.center = (x_0[i], y_0[i])
        circle2.center = (x_0[i], y_0[i])
        point0.set_data(x_0[i], y_0[i])  
        point1.set_data(x_1[i], y_1[i])   
        point2.set_data(x_2[i], y_2[i])
        point_cm.set_data(x_cm[i], y_cm[i])
        line1.set_data([x_0[i], x_1[i]], [y_0[i], y_1[i]])
        line2.set_data([x_1[i], x_2[i]], [y_1[i], y_2[i]])
        line3.set_data(x_2[:i], y_2[:i])
        line_cm.set_data(x_cm[:i], y_cm[:i])
        trans = Affine2D()
        u_arrow._patch_transform = trans.scale(U[i,0] * arrow_l / u_max, arrow_w).translate(x_0[i],0)
        for ii in range(len(texts)):
            text = texts[ii]
            name = print_var_names[ii]
            arr = print_vars[ii]
            text.set_text("$" + name + "$ = %.6f" % arr[i])
        return (line1,line2,)
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=N, interval=20, 
                               blit=True)
    return anim

In [ ]:
F_cas = RHS2casF(RHS, 3)

In [ ]:
F_cas([1,0,1,1,1,1], [0,0,0], [1,1,1,1,1,1,1])

In [ ]:
F_cas

In [ ]:
F_nump = RHS2numpy(RHS, 3)

In [ ]:
F_nump([1,0,1,1,1,1], [0,0,0], [1,1,1,1,1,1,1])

In [ ]:
F_nump

In [ ]:
from optibot.symbolic import diff_to_symb

E=T+U

print_funcs([diff_to_symb(E, 3),], 3)

In [ ]:
from optibot.numpy import unpack
def energ(x,params):
    q_0, q_1, q_2, v_0, v_1, v_2 = unpack(x)
    g, k, l_1, l_2, m_0, m_1, m_2 = params
    result = g*(-l_1*m_1*np.cos(q_1) + m_2*(-l_1*np.cos(q_1) - l_2*np.cos(q_2))) + k*(q_1 - q_2)**2/2 + m_0*v_0**2/2 + m_1*(l_1**2*v_1**2 + 2*l_1*v_0*v_1*np.cos(q_1) + v_0**2)/2 + m_2*(l_1**2*v_1**2 + 2*l_1*l_2*v_1*v_2*np.cos(q_1 - q_2) + 2*l_1*v_0*v_1*np.cos(q_1) + l_2**2*v_2**2 + 2*l_2*v_0*v_2*np.cos(q_2) + v_0**2)/2
    
    return result

In [ ]:
new_X = np.zeros([(2*N)+1,6],dtype = 'float')
new_X[:-1,0]= new_X_interp[:,0]
new_X[:-1,1]= new_X_interp[:,1]
new_X[:-1,2]= new_X_interp[:,1]
new_X[:-1,3]= new_X_interp[:,2]
new_X[:-1,4]= new_X_interp[:,3]
new_X[:-1,5]= new_X_interp[:,3]
new_U = np.zeros([(2*N)+1,3],dtype = 'float')
new_X[:-1,0]= new_X_interp[:,0]

In [ ]:
opti = cas.Opti()
opti.solver('ipopt')

In [ ]:
X = opti.variable((2*N)+1,6)
U = opti.variable((2*N)+1,3)
T = opti.parameter()
H = T/(2*N)
u_m = opti.parameter(3)

In [ ]:
#M = opti.parameter(2)
#L = opti.parameter(2)
#G = opti.parameter()
Params = opti.parameter(7)

In [ ]:
#cost = cas.sum1(2+cas.cos(X[:,0])+cas.cos(X[:,1]))*T
cost = cas.sum1(cas.cos(X[:,1])) + cas.sum1(X[:,0]**2) + cas.sum1((X[:,1]-X[:,2])**2)
#cost = cas.sum1(cas.cos(X[:,1]))
opti.minimize(cost)

In [ ]:
opti.subject_to(X[0,:].T == [0, 0, 0, 0, 0, 0])
#opti.subject_to(opti.bounded(-0.0001,X[-1,0],0.0001))
opti.subject_to(cas.cos(X[-1,1]) < -0.9999)
#opti.subject_to(cas.cos(X[-1,2]) < -0.9999)
opti.subject_to(opti.bounded(-0.0001,X[-1,3],0.0001))
#opti.subject_to(opti.bounded(-0.0001,X[-1,4],0.0001))
#opti.subject_to(opti.bounded(-0.0001,X[-1,5],0.0001))
opti.subject_to(U[-1,:].T == [0, 0, 0])

In [ ]:
#restriction = restriction2casadi(euler_restr, F_cas, 3, 7)
restriction = restriction2casadi(trapz_restr, F_cas, 3, 7)
#restriction = restriction2casadi(hs_restr, F_cas, 3, 7)

In [ ]:
for ii in range(2*N):
    opti.subject_to(restriction(X[ii,:], X[ii+1,:], U[ii,:], U[ii+1,:],H, Params)==0)
    opti.subject_to(opti.bounded(-u_m[0],U[ii, 0],u_m[0]))
    opti.subject_to(opti.bounded(-u_m[1],U[ii, 1],u_m[1]))
    opti.subject_to(opti.bounded(-u_m[2],U[ii, 2],u_m[2]))

In [ ]:
opti.set_initial(X[:,1], np.linspace(0, cas.pi, (2*N)+1))
opti.set_initial(X[:,2], np.linspace(0, cas.pi, (2*N)+1))
opti.set_initial(X[:,4], cas.pi/(2*N))
opti.set_initial(X[:,5], cas.pi/(2*N))
opti.set_initial(U, np.zeros([(2*N)+1,3],dtype = 'float'))

In [ ]:
opti.set_initial(X, new_X)
opti.set_initial(U, new_U)

In [ ]:
opti.set_value(T, 12)
max_par_1 = 0.0
max_par_2 = 0.0
max_f = 1.0
opti.set_value(u_m, [max_f, max_par_1, max_par_2])

m_0, m_1, m_2 = [0.2, 0.2, 0.2]
l_1, l_2 = [0.2, 0.2]
k = 5
g = 9.8

opti.set_value(Params, [g, k, l_1, l_2, m_0, m_1, m_2])

In [ ]:
sol = opti.solve()

In [ ]:
xx = sol.value(X)
uu = sol.value(U)
ene = energ(xx, [g, k, l_1, l_2, m_0, m_1, m_2])
ene = np.array(ene)
pot = np.sum(uu[:,:] * xx[:,3:], axis=1) 

In [ ]:

work = trapz_integrate(pot, np.linspace(0, sol.value(T), (2*N)+1)) + ene[0] 

In [ ]:
plt.figure(figsize=[14,10])
plt.plot(xx[:,0], 'b')
plt.plot(xx[:,2], 'b:')
plt.plot(xx[:,1], 'orange')
plt.plot(xx[:,3], 'orange', ls = ':')
plt.plot(uu[:,0], 'g:')
plt.plot(uu[:,1], 'r:')

In [ ]:
plt.figure(figsize=[14,10])
#plt.plot(xx[:,2], 'orange')
plt.plot(ene, 'r:')
#plt.plot(uu[:,0], 'g:')
#plt.plot(uu[:,0] * xx[:,2], 'b:')
plt.plot(work, 'b-', linewidth = 1)
#plt.hlines([0,np.pi, -np.pi], 0, N, 'k', 'dotted')

In [ ]:
pot_ene = []
for ii in range(len(ene)-1):
    pot_ene.append(ene[ii+1]-ene[ii])
pot_ene = np.array(pot_ene)

In [ ]:
pot_ene = num_derivative(ene, sol.value(T) / (2*N))

In [ ]:
plt.figure(figsize=[14,10])
#plt.plot(xx[:,2], 'orange')
#plt.plot(ene.flatten(), 'r:')
plt.plot(pot, 'g:')
plt.plot(pot_ene, 'r:')
#plt.plot(-uu[:,0] * xx[:,2], 'b:')
#plt.plot(work, 'b-', linewidth = 1)
plt.hlines([0,], 0, N, 'k', 'dotted')

In [ ]:
new_X = xx
new_U = uu

In [ ]:
m_0, m_1, m_2 = [0.2, 0.2, 0.2]
l_1, l_2 = [0.2, 0.2]
k = 5
g = 9.8

anim = create_anim(xx,uu,[g, k, l_1, l_2, m_0, m_1, m_2])

In [ ]:
HTML(anim.to_jshtml())

In [ ]:
m_0, m_1, m_2 = [0.2, 0.2, 0.2]
l_1, l_2 = [0.2, 0.2]
k = 5
g = 9.8
dt = 0.005

In [ ]:
uu_prueba = np.zeros([200, 3])
xx_prueba_eu = integrate_euler(
    np.array([0, 1, 1.1,0,0,0]),
    uu_prueba,
    F_nump,
    dt,
    [g, k, l_1, l_2, m_0, m_1, m_2]
)

In [ ]:
anim_prueba_eu = create_anim(np.array(xx_prueba_eu)[:-1],uu_prueba,[g, k, l_1, l_2, m_0, m_1, m_2])

In [ ]:
HTML(anim_prueba_eu.to_jshtml())

In [ ]:
m_0, m_1, m_2 = [0.2, 0.2, 0.2]
l_1, l_2 = [0.2, 0.2]
k = 25
g = 9.8
dt = 0.005

In [ ]:
uu_prueba = np.zeros([200, 3])
xx_prueba = integrate_hs(
    np.array([0, 1, 1.1,0,0,0]),
    uu_prueba,
    F_nump,
    dt,
    [g, k, l_1, l_2, m_0, m_1, m_2]
)

In [ ]:
anim_prueba = create_anim(np.array(xx_prueba)[:-1],uu_prueba,[g, k, l_1, l_2, m_0, m_1, m_2])

In [ ]:
HTML(anim_prueba.to_jshtml())